# Robust PCA

    Solve the robust PCA problem taken from Yi, Xinyang, et al. "Fast algorithms for robust PCA via gradient
    descent." Advances in neural information processing systems. 2016.


    If you publish work that uses or refers to PyGRANSO, please cite both
    PyGRANSO and GRANSO paper:

    [1] Buyun Liang, Tim Mitchell, and Ju Sun,
        NCVX: A User-Friendly and Scalable Package for Nonconvex
        Optimization in Machine Learning, arXiv preprint arXiv:2111.13984 (2021).
        Available at https://arxiv.org/abs/2111.13984

    [2] Frank E. Curtis, Tim Mitchell, and Michael L. Overton,
        A BFGS-SQP method for nonsmooth, nonconvex, constrained
        optimization and its evaluation using relative minimization
        profiles, Optimization Methods and Software, 32(1):148-181, 2017.
        Available at https://dx.doi.org/10.1080/10556788.2016.1208749

    ex4_robustPCA.ipynb (introduced in PyGRANSO v1.0.0)
    Copyright (C) 2021 Buyun Liang

    New code and functionality for PyGRANSO v1.0.0.

    For comments/bug reports, please visit the PyGRANSO webpage:
    https://github.com/sun-umn/PyGRANSO

    PyGRANSO Version 1.0.0, 2021, see AGPL license info below.

    =========================================================================
    |  PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation |
    |  Copyright (C) 2021 Tim Mitchell and Buyun Liang                      |
    |                                                                       |
    |  This file is part of PyGRANSO.                                       |
    |                                                                       |
    |  PyGRANSO is free software: you can redistribute it and/or modify     |
    |  it under the terms of the GNU Affero General Public License as       |
    |  published by the Free Software Foundation, either version 3 of       |
    |  the License, or (at your option) any later version.                  |
    |                                                                       |
    |  PyGRANSO is distributed in the hope that it will be useful,          |
    |  but WITHOUT ANY WARRANTY; without even the implied warranty of       |
    |  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the        |
    |  GNU Affero General Public License for more details.                  |
    |                                                                       |
    |  You should have received a copy of the GNU Affero General Public     |
    |  License along with this program.  If not, see                        |
    |  <http://www.gnu.org/licenses/agpl.html>.                             |
    =========================================================================

## Problem Description

$$\min_{M,S}||M||_{\text{nuc}}+\lambda||S||_1,$$
$$\text{s.t. }Y=M+S,$$
where $M,S\in R^{d_1,d_2}$ are matrix form optimization variables, $Y\in R^{d_1,d_2}$ is a given matrix, and $||\cdot||_{\text{nuc}}$ denotes the nuclear norm.

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
import sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/PyGRANSO')
from pygranso import pygranso
from pygransoStruct import pygransoStruct

## Data Generation 
Specify torch device, and generate data.

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [2]:
device = torch.device('cuda')
d1 = 3
d2 = 4
torch.manual_seed(1)
eta = .05
Y = torch.randn(d1,d2).to(device=device, dtype=torch.double)

## Problem Definition


Specify optimization variables, and objective and constraint(s).

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [3]:
# variables and corresponding dimensions.
var_in = {"M": [d1,d2],"S": [d1,d2]}


def comb_fn(X_struct):
    M = X_struct.M
    S = X_struct.S
    
    # objective function
    f = torch.norm(M, p = 'nuc') + eta * torch.norm(S, p = 1)

    # inequality constraint, matrix form
    ci = None
    
    # equality constraint 
    ce = pygransoStruct()
    ce.c1 = M + S - Y

    return [f,ci,ce]


## User Options
Specify user-defined options for PyGRANSO

In [4]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 10
opts.x0 = .2 * torch.ones((2*d1*d2,1)).to(device=device, dtype=torch.double)
opts.opt_tol = 1e-6

## Main Algorithm

In [5]:
start = time.time()
soln = pygranso(var_spec = var_in,combined_fn = comb_fn,user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.0.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021 Tim Mitchell and Buyun Liang                                       ║ 
═════════════════════════════════════════════